In [1]:
from ver_fotos_enpath import get_names
import tensorflow as tf
from bs4 import BeautifulSoup
from pathlib import Path

In [2]:
def check_outputhtml(path):

    if Path('output.html').exists(): 
        with open("output.html", "r") as html:
            soup = BeautifulSoup(html, 'html.parser')
            heads = [tag.string for tag in soup.find_all('h1')] 
            links = [l for l in soup.find_all('a')]    
            
            
        used_path = path in heads
        new_output = False
    else:    
        used_path = False
        new_output = True
        links = []
        soup = None
    num_of_fotos = len(links)  
    return used_path, new_output, soup, num_of_fotos
    

In [10]:
def process_pics(path, num_ofpics):
    tmp_output = ''
    names = get_names(path)
    for x, i in enumerate(names):
        foto = tf.io.read_file(i)
        foto = tf.io.decode_jpeg(foto)
        foto = tf.image.convert_image_dtype(foto, tf.float32)
        foto = tf.image.resize(foto, [128, 128], preserve_aspect_ratio=True)
        img = tf.keras.preprocessing.image.array_to_img(foto)
        img_string = "/Users/dealeon/Directorio_fotostf/fotito" + str(x + num_ofpics) + ".jpg"
        img.save(img_string)  
        i = i.replace(" ", "%20") #subdirectories (i) migh have spaces, links get broken if not replaced with %20
        
        tmp_output += '<a href=' + i  +'><img src="' + img_string + '"></img></a>\n'
        
    return tmp_output        


In [4]:

def build_html(path, used_path, new_output, soup, num_ofpics):

    if used_path: return
    
    output = ''
    if new_output:
        
        output = '<!DOCTYPE html>\n'
        output+= '<html>\n'
        output+= '<head></head>\n'
        output+= '<body>\n'
        output+= '<h1>' + path + '</h1>'
        output+= process_pics(path, num_ofpics) 
        output+= '</body>\n'
        output+= '</html>\n'
        with open("output.html", "w") as new_html:
            new_html.write(output)

    else:
        tag = soup.new_tag("h1")
        tag.string = path
        original_body=soup.body
        original_body.append(tag)
        tmp_output = process_pics(path, num_ofpics)
        soup2 = BeautifulSoup(tmp_output)
        soup.body.append(soup2)
        with open("output.html", "w") as f:
                    f.write(str(soup))
    return            

In [13]:
#path = "/Users/dealeon/Pictures/Fotos/Noruega2009"
#path = "/Users/dealeon/Dir_de_prueba"
path = "/Users/dealeon/Pictures/Fotos - library/2019"
#path = "/Users/dealeon/Pictures/Fotos - library/22 June 2015"
#path = "/Users/dealeon/Pictures/Fotos - library/5 April 2015"
output_file = "output.html"

if Path(path).exists():
    used_path, new_output, soup, num_ofpics = check_outputhtml(path)

    build_html(path, used_path, new_output, soup, num_ofpics)

In [6]:
#print(soup.prettify())